In [1]:
import os
import csv
import pandas as pd
from pinecone import Pinecone, PodSpec
from langchain_pinecone import PineconeVectorStore
from langchain.indexes import SQLRecordManager, index
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

e:\Movie Search\Movie-Bot-Anantadi\app\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_KEY =  "b3d91009-fae8-4f09-b32d-7959b043996a"
PINECONE_INDEX_NAME =  'movie'
OPENAI_KEY =  "sk-proj-wjhlw45dvsRqQyuMMERWT3BlbkFJvS1rCNcHX0qScJcE7WSx"

In [11]:


# Initialize Pinecone with API key
pc = Pinecone(api_key=PINECONE_KEY)

# Create Pinecone index if not exists
# pc.create_index(
#     name=PINECONE_INDEX_NAME,
#     dimension=1536,
#     metric="cosine",
#     spec=PodSpec(
#         environment="gcp-starter"
#     )
# )

# Target the existing index and check its status
pc_index = pc.Index(PINECONE_INDEX_NAME)
print(pc_index.describe_index_stats())

# Initialize OpenAI embeddings with API key
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPENAI_KEY)

# Read CSV file
csv_file_path = "newtmdb.csv"
df = pd.read_csv(csv_file_path)

# Define the documents list
docs = []
for index, row in df.iterrows():
    document = Document(
        page_content=row["overview"],  # Assuming "overview" column contains text content
        metadata={
            "title": row["title"],  # Assuming "title" column contains movie title
            "genre": row["genre_ids"],  # Assuming "genre_ids" column contains genre IDs
            "vote_average": row["vote_average"],  # Assuming "vote_average" column contains average vote
            "release_date": row["release_date"],  # Assuming "release_date" column contains release date
            # Add more metadata fields as needed
        }
    )
    docs.append(document)

# Create Pinecone vector store from documents
vectorstore = PineconeVectorStore.from_documents(
    docs, embeddings, index_name=PINECONE_INDEX_NAME
)

print("Pinecone vector store created successfully.")

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4549}},
 'total_vector_count': 4549}
Pinecone vector store created successfully.


In [13]:
import os
import csv
import pandas as pd
from pinecone import Pinecone, PodSpec
from langchain_pinecone import PineconeVectorStore
from langchain.indexes import SQLRecordManager, index
from langchain_openai import OpenAIEmbeddings

# Load environment variables
PINECONE_KEY = "b3d91009-fae8-4f09-b32d-7959b043996a"
PINECONE_INDEX_NAME =  'movie'
OPENAI_KEY =  "sk-proj-wjhlw45dvsRqQyuMMERWT3BlbkFJvS1rCNcHX0qScJcE7WSx"


In [14]:
from langchain.chains.query_constructor.base import AttributeInfo
 

metadata_field_info = [
    AttributeInfo(
        name="title",
        description="The title of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="genre_ids",
        description="Genre ids of the movie",
        type="integer",
    ),
    AttributeInfo(
        name="vote_average",
        description="A 1-10 rating for the movie",
        type="float",
    ),
    AttributeInfo(
        name="release_date", description="Release Date of the movie", type="string"
    ),
]
document_content_description = "Brief summary of a movie"
# llm = OpenAI(temperature=0)
# retriever = SelfQueryRetriever.from_llm(
#     llm, vectorstore, document_content_description, metadata_field_info 
# )

In [15]:
from langchain_openai import (
    ChatOpenAI, 
    OpenAIEmbeddings
)
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
    AttributeInfo
)

In [18]:
%pip install --upgrade --quiet  lark

Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install --upgrade --quiet  pinecone-client

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.1.1 requires pinecone-client<4.0.0,>=3.2.2, but you have pinecone-client 4.1.0 which is incompatible.


In [16]:
import lark
document_content_description = "Brief overview of a movie, along with keywords"

# Define allowed comparators list
allowed_comparators = [
    "$eq",  # Equal to (number, string, boolean)
    "$ne",  # Not equal to (number, string, boolean)
    "$gt",  # Greater than (number)
    "$gte",  # Greater than or equal to (number)
    "$lt",  # Less than (number)
    "$lte",  # Less than or equal to (number)
    "$in",  # In array (string or number)
    "$nin",  # Not in array (string or number)
]

examples = [
    (
        "I'm looking for a sci-fi comedy released after 2021.",
        {
            "query": "sci-fi comedy",
            "filter": "and(eq('Genre', 'Science'), eq('Genre', 'Comedy'), gt('Release Year', 2021))",
        },
    ),
    (
        "Show me critically acclaimed dramas without Tom Hanks.",
        {
            "query": "critically acclaimed drama",
            "filter": "and(eq('Genre', 'Drama'), nin('Actors', ['Tom Hanks']))",
        },
    ),
    (
        "Recommend some films by Yorgos Lanthimos.",
        {
            "query": "Yorgos Lanthimos",
            "filter": 'in("Directors", ["Yorgos Lanthimos]")',
        },
    ),
    (
        "Films similar to Yorgos Lanthmios movies.",
        {
            "query": "Dark comedy, absurd, Greek Weird Wave",
            "filter": 'NO_FILTER',
        },
    ),
    (
        "Find me thrillers with a strong female lead released between 2015 and 2020.",
        {
            "query": "thriller strong female lead",
            "filter": "and(eq('Genre', 'Thriller'), gt('Release Year', 2015), lt('Release Year', 2021))",
        },
    ),
    (
        "Find me highly rated drama movies in English that are less than 2 hours long",
        {
            "query": "Highly rated drama English under 2 hours",
            "filter": 'and(eq("Genre", "Drama"), eq("Language", "English"), lt("Runtime (minutes)", 120))',
        },
    ),
]

constructor_prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
    allowed_comparators=allowed_comparators,
    examples=examples,
)
openai_key = "sk-proj-wjhlw45dvsRqQyuMMERWT3BlbkFJvS1rCNcHX0qScJcE7WSx"
query_model = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    temperature=0,
    streaming=True,api_key=openai_key
)

output_parser = StructuredQueryOutputParser.from_components()
query_constructor = constructor_prompt | query_model | output_parser

In [22]:
question = "Godzilla"
query_constructor.invoke(
    {
        "query": question
    }
)

StructuredQuery(query='Godzilla', filter=None, limit=None)

In [23]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers.self_query.pinecone  import PineconeTranslator
retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore,
    structured_query_translator=PineconeTranslator(),
    search_kwargs={'k': 10}
)

retriever.invoke(question)

[Document(page_content='Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', metadata={'genre': '[878, 28, 12]', 'release_date': '2024-03-27', 'title': 'Godzilla x Kong: The New Empire', 'vote_average': 7.256}),
 Document(page_content='Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', metadata={'genre': '[878, 28, 12]', 'release_date': '2024-03-27', 'title': 'Godzilla x Kong: The New Empire', 'vote_average': 7.256}),
 Document(page_content='Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', metadata={'genre': '[878, 28, 12]', 'release_date': '2024-03-27', 'title': 'Godzilla x Kong: The New Empire', 'vote

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [28]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

In [35]:

def format_docs(docs):
    return "\n\n".join(f"{doc.page_content}\n\nMetadata: {doc.metadata}" for doc in docs)

chat_model = ChatOpenAI(
    model='gpt-3.5-turbo-0125',
    # model='gpt-4-0125-preview',
    temperature=0,
    streaming=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """
            Your goal is to recommend films to users based on their 
            query and the retrieved context. If a retrieved film doesn't seem 
            relevant, omit it from your response. Never refer to films that
            are not in your context. If you cannot recommend any 
            films, suggest better queries to the user. You cannot 
            recommend more than five films. Your recommendation should 
            be relevant, original, and at least two to three sentences 
            long.
            
            YOU CANNOT RECOMMEND A FILM IF IT DOES NOT APPEAR IN YOUR 
            CONTEXT.

            # TEMPLATE FOR OUTPUT
            - [Title of Film](source link):
                - Runtime:
                - Release Year:
                - (Your reasoning for recommending this film)
            
            Question: {question} 
            Context: {context} 
            """
        ),
    ]
)

# Create a chatbot Question & Answer chain from the retriever
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | chat_model
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)


query_constructor.invoke(
    {
        "query": question
    }
)
# Only prints final answer
# for chunk in rag_chain_with_source.stream(question):
#     for key in chunk:
#         if key == 'answer':
#             print(chunk[key], end="", flush=True)

# Prints everything
output = {}
curr_key = None
for chunk in rag_chain_with_source.stream(question):
    for key in chunk:
        if key not in output:
            output[key] = chunk[key]
        else:
            output[key] += chunk[key]
        if key != curr_key:
            print(f"\n\n{key}: {chunk[key]}", end="", flush=True)
        else:
            print(chunk[key], end="", flush=True)
        curr_key = key
output



question: Godzilla

context: [Document(page_content='Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', metadata={'genre': '[878, 28, 12]', 'release_date': '2024-03-27', 'title': 'Godzilla x Kong: The New Empire', 'vote_average': 7.256}), Document(page_content='Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', metadata={'genre': '[878, 28, 12]', 'release_date': '2024-03-27', 'title': 'Godzilla x Kong: The New Empire', 'vote_average': 7.256}), Document(page_content='Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', metadata={'genre': '[878, 28, 12]', 'release_date': '2024-03-27', 'title': 'Godzilla x

{'question': 'Godzilla',
 'context': [Document(page_content='Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', metadata={'genre': '[878, 28, 12]', 'release_date': '2024-03-27', 'title': 'Godzilla x Kong: The New Empire', 'vote_average': 7.256}),
  Document(page_content='Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', metadata={'genre': '[878, 28, 12]', 'release_date': '2024-03-27', 'title': 'Godzilla x Kong: The New Empire', 'vote_average': 7.256}),
  Document(page_content='Following their explosive showdown, Godzilla and Kong must reunite against a colossal undiscovered threat hidden within our world, challenging their very existence – and our own.', metadata={'genre': '[878, 28, 12]', 'release_date': '2024-03-27', 'title': '

In [34]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv("OPENAI_API_KEY")

In [32]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
